### 13.1.1 单辆车的TSP问题

从北京出发，经过其他城市，且仅一次

In [1]:
import pandas as pd
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

#### 1 最简单的tsp类

In [2]:
class tsp:
    def __init__(self):
        #设置城市名
        self.df = pd.read_excel("./cities.xlsx")
        
    def create_data_model(self):
        data = {}
        temp = pd.read_excel("./distance.xlsx", index_col=0)
#         print(temp)
        data["distance_matrix"] = temp.values / 100
    
        data['num_vehicles'] = 1
        data['depot'] = 0
        return data
    
    def print_solution(self, manager, routing, solution):
#         print('总行驶里程: {} 公里'.format(solution.ObjectiveValue()))
        index = routing.Start(0)
        route = []
        route_distance = 0
        while not routing.IsEnd(index):
            index = manager.IndexToNode(index)
            route.append(index)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
        return route, route_distance
    
    def work(self):
        # 初始化数据
        data = self.create_data_model()

        # 创建路线管理，tsp_size（城市数量）, num_vehicles（车的数量）, depot（原点）
        manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                               data['num_vehicles'], data['depot'])

        # 创建 Routing Model.
        routing = pywrapcp.RoutingModel(manager)

        # 计算两点之间的距离
        def distance_callback(from_index, to_index):
            # Convert from routing variable Index to distance matrix NodeIndex.
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            return data['distance_matrix'][from_node][to_node]

        transit_callback_index = routing.RegisterTransitCallback(distance_callback)

        # Define cost of each arc.
        routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

        # Setting first solution heuristic.
        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        search_parameters.first_solution_strategy = (
            routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

        # 求解路径规划
        solution = routing.SolveWithParameters(search_parameters)
        # 输出结果
        route, route_distance = self.print_solution(manager, routing, solution)
        return route, route_distance

In [3]:
model = tsp()
route, route_distance = model.work()
print(route)
print(route_distance)

[0, 6, 22, 21, 23, 24, 26, 27, 25, 12, 32, 11, 31, 30, 10, 9, 8, 15, 13, 28, 29, 14, 20, 17, 18, 19, 7, 5, 16, 1, 4, 2, 3]
198151


#### 2 加入city_names的tsp类(只去部分城市)

In [4]:
class tsp:
    def __init__(self, city_names=None):
        #设置城市名
        self.df = pd.read_excel("./cities.xlsx")
        self.all_city = self.df["name"].values
        if city_names is not None:
            self.city_names = city_names
            self.df = self.df[self.df["name"].isin(city_names)]
        else:
            self.city_names = self.all_city
        
    def create_data_model(self):
        data = {}
        temp = pd.read_excel("./distance.xlsx", index_col=0)
        # 按照self.city_names进行筛选
        temp = temp[temp.index.isin(self.city_names)][self.city_names]
#         print(temp)

        data["distance_matrix"] = temp.values / 1000
    
        data['num_vehicles'] = 1
        data['depot'] = 0
        return data
    
    def print_solution(self, manager, routing, solution):
#         print('总行驶里程: {} 公里'.format(solution.ObjectiveValue()))
        index = routing.Start(0)
        route = []
        route_distance = 0
        while not routing.IsEnd(index):
            index_show = manager.IndexToNode(index)
            route.append(index_show)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
        return route, route_distance
    
    def work(self):
        # 初始化数据
        data = self.create_data_model()

        # 创建路线管理，tsp_size（城市数量）, num_vehicles（车的数量）, depot（原点）
        manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                               data['num_vehicles'], data['depot'])

        # 创建 Routing Model.
        routing = pywrapcp.RoutingModel(manager)

        # 计算两点之间的距离
        def distance_callback(from_index, to_index):
            # Convert from routing variable Index to distance matrix NodeIndex.
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            return data['distance_matrix'][from_node][to_node]

        transit_callback_index = routing.RegisterTransitCallback(distance_callback)

        # Define cost of each arc.
        routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

        # Setting first solution heuristic.
        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        search_parameters.first_solution_strategy = (
            routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

        # 求解路径规划
        solution = routing.SolveWithParameters(search_parameters)
        # 输出结果
        route, route_distance = self.print_solution(manager, routing, solution)
        return route, route_distance

In [5]:
city_name = ["北京", "天津", "南京"]
model = tsp(city_name)
route, route_distance = model.work()
print(route)
print(route_distance)

[0, 1, 2]
2059


### 13.1.2 多辆车的路径规划VRP

In [15]:
class tsp:
    def __init__(self, city_names=None, num_vehicles=1):
        #设置城市名
        self.df = pd.read_excel("./cities.xlsx")
        self.all_city = self.df["name"].values
        self.num_vehicles = num_vehicles
        if city_names is not None:
            self.city_names = city_names
            self.df = self.df[self.df["name"].isin(city_names)]
        else:
            self.city_names = self.all_city
        
    def create_data_model(self):
        data = {}
        temp = pd.read_excel("./distance.xlsx", index_col=0)
        # 按照self.city_names进行筛选
        temp = temp[temp.index.isin(self.city_names)][self.city_names]
#         print(temp)

        data["distance_matrix"] = temp.values / 1000
    
        data['num_vehicles'] = self.num_vehicles
        data['depot'] = 0
        return data
    
    def get_solution(self, manager, routing, solution):
#         print('总行驶里程: {} 公里'.format(solution.ObjectiveValue()))
        # 记录每辆车的里程
        distance_list = []
        # 记录每辆车的路径规划
        route_list = []
        
        # 遍历每辆车
        for vehicle_id in range(self.num_vehicles):
            # 初始化距离为0
            route_distance = 0
            route = []
            # 从vehicle_id的起始点出发
            index = routing.Start(vehicle_id)
            
            
            while not routing.IsEnd(index):
                index_show = manager.IndexToNode(index)
                route.append(index_show)
                previous_index = index
                index = solution.Value(routing.NextVar(index))
                route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
                
            route_list.append(route)
            distance_list.append(route_distance)
        return route_list, distance_list
    
     # 添加距离约束
    def add_distance_dimension(self, routing, transit_callback_index):
        
        dimension_name = 'Distance'
        routing.AddDimension(
            transit_callback_index,
            0,  # no slack
            10000,  # 车辆最大行驶距离
            True,  # start cumul to zero
            dimension_name)
        distance_dimension = routing.GetDimensionOrDie(dimension_name)
        # 尽量减少车辆之间的最大距离
        distance_dimension.SetGlobalSpanCostCoefficient(100)

    
    def work(self):
        # 初始化数据
        data = self.create_data_model()

        # 创建路线管理，tsp_size（城市数量）, num_vehicles（车的数量）, depot（原点）
        manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                               data['num_vehicles'], data['depot'])

        # 创建 Routing Model.
        routing = pywrapcp.RoutingModel(manager)

        # 计算两点之间的距离
        def distance_callback(from_index, to_index):
            # Convert from routing variable Index to distance matrix NodeIndex.
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            return data['distance_matrix'][from_node][to_node]

        transit_callback_index = routing.RegisterTransitCallback(distance_callback)

        # Define cost of each arc.
        routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
        
        # 添加距离约束
        self.add_distance_dimension(routing, transit_callback_index)
        # Setting first solution heuristic.
        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        search_parameters.first_solution_strategy = (
            routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

        # 求解路径规划
        solution = routing.SolveWithParameters(search_parameters)
        # 输出结果
        route, route_distance = self.get_solution(manager, routing, solution)
        return route, route_distance

In [16]:
city_name = ["北京", "天津", "南京"]
model = tsp(num_vehicles=4)
route, route_distance = model.work()
print(route)
print(route_distance)

[[0, 7, 18, 19, 17, 22, 6], [0, 21], [0, 5, 20, 23, 24, 26, 27, 25, 12, 32, 11, 31, 30, 13, 16], [0, 14, 28, 29, 10, 9, 8, 15, 1, 4, 2, 3]]
[6341, 7096, 6749, 6845]
